# Visualize suspected RBCs

#### This notebook labels each cell within this patient's images by it's Phenograph cluster number generated in Phenograph-RGB_features.ipynb. Suspected RBCs are outlined in red to visually confirm.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile
import cv2
from PIL import Image
from PIL import ImageDraw
from skimage import data, color, io, img_as_float
from os import listdir
from os.path import isfile, join
import skimage.io
import skimage.util
from skimage import data, segmentation, util, measure
from skimage.segmentation import expand_labels

In [2]:
# create dictionary of directories for each patient
img_dirs = {'P51':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P51_R1158_S1_Normalized/', 
            'P52':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P52_R1149_S1_Normalized/', 
            'P53':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P53_R1149_S2_Normalized/', 
            'P56':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P56_R1158_S2_Normalized/', 
            'P57':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P57_R1158_S3_Normalized/', 
            'P58':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P58_R1149_S3_Normalized/'}

In [17]:
patient = 'P57'
tp = 'A'

In [18]:
# create dictionary of directories for each patient
mask_dir = '/data/Zhaolab/1_AMLCosMx/Final_scripts/2_Segmentation/3_NucMemMerging/' + patient + '_hybrid/labels_predicted_2_15_24/'

In [25]:
# load in celltype predictions for all cells
morph_predicted = pd.read_csv(patient + tp + '_RGB_clusters.csv')
morph_predicted

,Unnamed: 0,FOV,cell_ID,Timepoint,PhenoGraph_clusters
0,P57_FOV01_cell_1,1,1,A,0
1,P57_FOV01_cell_2,1,2,A,7
2,P57_FOV01_cell_3,1,3,A,0
3,P57_FOV01_cell_4,1,4,A,7
4,P57_FOV01_cell_5,1,5,A,6
...,...,...,...,...,...
26485,P57_FOV07_cell_4705,7,4705,A,18
26486,P57_FOV07_cell_4706,7,4706,A,11
26487,P57_FOV07_cell_4707,7,4707,A,14
26488,P57_FOV07_cell_4708,7,4708,A,14


In [26]:
morph_predicted = morph_predicted.drop(columns=["Unnamed: 0"])
morph_predicted

,FOV,cell_ID,Timepoint,PhenoGraph_clusters
0,1,1,A,0
1,1,2,A,7
2,1,3,A,0
3,1,4,A,7
4,1,5,A,6
...,...,...,...,...
26485,7,4705,A,18
26486,7,4706,A,11
26487,7,4707,A,14
26488,7,4708,A,14


In [27]:
# add col to indicate if cell is predicted RBC
morph_predicted['RBC'] = 0
morph_predicted['RBC'][morph_predicted['PhenoGraph_clusters'].isin([1,2,5,8,9,11,12,13,16,18])] = 1
morph_predicted

/tmp/ipykernel_3147655/4023331249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morph_predicted['RBC'][morph_predicted['PhenoGraph_clusters'].isin([1,2,5,8,9,11,12,13,16,18])] = 1


,FOV,cell_ID,Timepoint,PhenoGraph_clusters,RBC
0,1,1,A,0,0
1,1,2,A,7,0
2,1,3,A,0,0
3,1,4,A,7,0
4,1,5,A,6,0
...,...,...,...,...,...
26485,7,4705,A,18,1
26486,7,4706,A,11,1
26487,7,4707,A,14,0
26488,7,4708,A,14,0


In [28]:
morph_predicted[morph_predicted['RBC'] == 1]

,FOV,cell_ID,Timepoint,PhenoGraph_clusters,RBC
5,1,6,A,8,1
17,1,18,A,8,1
19,1,20,A,8,1
31,1,32,A,5,1
32,1,33,A,11,1
...,...,...,...,...,...
26482,7,4701,A,9,1
26484,7,4704,A,9,1
26485,7,4705,A,18,1
26486,7,4706,A,11,1


In [29]:
# save list of all patient's RBCs
pos_to_save = morph_predicted[morph_predicted['RBC'] == 1][['FOV', 'cell_ID']]
pos_to_save['Patient'] = patient

### Visualize all cells with cluster names, highlight likely RBCs

In [30]:
# create list of fovs
fovs = list(set(morph_predicted['FOV'].tolist()))
fovs.sort()
    
for fov in fovs:
    
    # subset FOV        
    one_fov = morph_predicted[morph_predicted['FOV'] == fov]
    
    # Load img
    location = img_dirs[patient]
    dirlist = listdir(location)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            img = cv2.imread(location + item)
            print(item)
            break
            
    # brighten image
    # define the alpha and beta
    alpha = 1.5 # Contrast control
    beta = 20 # Brightness control

    # call convertScaleAbs function
    img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    # Load masks
    location = mask_dir
    dirlist = listdir(location)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            labels = tifffile.imread(location + item)
            print(item)
            break
        elif 'FOV' + str(fov).zfill(2) in item:
            labels = tifffile.imread(location + item)
            print(item)
            break

    # get all mask values
    fov_cells = np.unique(labels)
    
    # Load centroids
    coords_dir = mask_dir + 'cell_centroids/'
    dirlist = listdir(coords_dir)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            centroids = pd.read_csv(coords_dir + item, index_col=0)
            break
        elif 'FOV' + str(fov).zfill(2) in item:
            centroids = pd.read_csv(coords_dir + item, index_col=0)
            break
            
    # merge prediction and centroid tables
    one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)
    one_fov = one_fov.rename(columns={"cell_ID": "label"})
                
    combo = pd.merge(one_fov,centroids,left_on='label',right_on='label',how='left')
    
    # visualize all segmented cells
    boundaries = segmentation.find_boundaries(labels, connectivity=1, mode='inner', background=0)
    boundaries = boundaries.astype(int)
    img[boundaries>0] = [255,255,0]
    
    # create list of RBCs
    pos_cells = combo[combo['RBC'] == 1]['label'].tolist()
    pos_cells = [int(i) for i in pos_cells]
    # set all labels of other clusters to zero
    labels_edited = labels.copy()
    for cell in fov_cells:
        if cell in pos_cells:
            pass
        else:
            labels_edited[labels_edited == cell] = 0

    # add colored outline
    boundaries = segmentation.find_boundaries(labels_edited, connectivity=1, mode='thick', background=0)
    boundaries = boundaries.astype(int)
    img[boundaries>0] = [0,0,255]

    # get unique phenograph clusters
    clusters = list(set(combo['PhenoGraph_clusters'].tolist()))
    
    for cluster in clusters:

        # create list of this cluster's cells
        pos_cells_df = combo[combo['PhenoGraph_clusters'] == cluster]

        for k in range(pos_cells_df.shape[0]):
            img = cv2.putText(img, str(cluster), (int(pos_cells_df['x'].iloc[k]), int(pos_cells_df['y'].iloc[k])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255,255,255], 2, cv2.LINE_AA)
            
    # save image
    cv2.imwrite('RGB_clusters_overlaid/' + patient + '_FOV' + str(fov).zfill(2) + '_RGB_cluster.png', img)

20220228_110038_S3_C902_P99_N99_F001_normalized.png
20220228_110038_S3_C902_P99_N99_F001_normalized_cp_masks.tif


/tmp/ipykernel_3147655/3996469836.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220228_110038_S3_C902_P99_N99_F002_normalized.png
20220228_110038_S3_C902_P99_N99_F002_normalized_cp_masks.tif


/tmp/ipykernel_3147655/3996469836.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220228_110038_S3_C902_P99_N99_F003_normalized.png
20220228_110038_S3_C902_P99_N99_F003_normalized_cp_masks.tif


...

/tmp/ipykernel_3147655/3996469836.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220228_110038_S3_C902_P99_N99_F006_normalized.png
20220228_110038_S3_C902_P99_N99_F006_normalized_cp_masks.tif


/tmp/ipykernel_3147655/3996469836.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220228_110038_S3_C902_P99_N99_F007_normalized.png
20220228_110038_S3_C902_P99_N99_F007_normalized_cp_masks.tif


/tmp/ipykernel_3147655/3996469836.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


In [31]:
# save IDs of RBCs
pos_to_save.to_csv(patient + tp + '_RBCs.csv', index=False)